In [1]:
import pandas as pd
import h3


df = pd.read_csv("../data/raw/final_sales_data_CLEANED.csv", encoding="utf-8-sig")



C:\Users\ACER\AppData\Local\Temp\ipykernel_22888\4058894971.py:5: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/final_sales_data_CLEANED.csv", encoding="utf-8-sig")


In [2]:
pd.set_option('display.max_columns', None)
# print(df['building_type'].value_counts())
# df.info()
# df.head()

In [3]:

#gereksizleri tullayiram
df = df.drop(columns=["createdAt", "updatedAt", "building_type", "all_pictures", "completion_status","purpose", "hasProject"])
df.columns = [col.lower() for col in df.columns]

# NOT: Sütun sıralaması tüm veri zenginleştirme (enrichment) 
# işlemlerinden sonra en sonda toplu olarak yapılacaktır.

In [4]:
#Numerical olan, lakin categorical olmali olan sutunlari categorical edirem. Reactivate date xaric, o date typedi. 
df = df.astype({
    'id': 'str',
    'rooms': 'Int64',
})

for col in ['building_no', 'postal_code', 'additional_no', 'deed_number']:
    df[col] = pd.to_numeric(df[col], errors="coerce").round().astype('Int64').astype('str')
    df[col] = df[col].replace(['nan','<NA>'], '')
    # print(df[col].value_counts())


df['reactivatedat'] = pd.to_datetime(df['reactivatedat'])
# df.info()
# df.head()

In [5]:
df.info()
df.describe().round(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18315 entries, 0 to 18314
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      18315 non-null  object        
 1   name                    18314 non-null  object        
 2   price                   18315 non-null  float64       
 3   rooms                   18315 non-null  Int64         
 4   bath                    18315 non-null  int64         
 5   area_m2                 18315 non-null  float64       
 6   location                18315 non-null  object        
 7   lat                     18315 non-null  float64       
 8   long                    18315 non-null  float64       
 9   building_no             18315 non-null  object        
 10  amenities               18247 non-null  object        
 11  furnished               18315 non-null  object        
 12  reactivatedat           18315 non-null  dateti

,price,rooms,bath,area_m2,lat,long,reactivatedat,beds
count,18315.00,18315.0,18315.00,18315.00,18315.00,18315.00,18315,18315.00
mean,813356.27,3.71,2.18,340.54,23.18,42.98,2025-10-13 11:05:47.321867520,2.45
min,350.00,1.0,0.00,24.73,21.41,38.96,2025-01-12 00:00:00,0.00
25%,550000.00,3.0,0.00,122.99,21.59,39.16,2025-09-01 00:00:00,0.00
50%,750000.00,4.0,3.00,146.17,24.50,46.50,2025-10-24 00:00:00,3.00
75%,950000.00,4.0,3.00,208.57,24.66,46.63,2025-12-12 00:00:00,4.00
max,13000000.00,345.0,11.00,103104.38,25.00,46.98,2026-01-07 00:00:00,11.00
std,582611.34,2.86,1.56,1770.79,1.55,3.74,NaN,1.82


In [6]:
#numerical, categorical, identical ve zaman olaraq 4 yere bolurem
num_cols = ['price', 'rooms','beds','bath', 'area_m2', 'lat', 'long']

cat_cols = ['location', 'amenities', 'furnished', 'city', 'region', 
                    'district', 'type', 'listing_age', 'listing_face', 
                    'is_listing_constrained', 'is_listing_pawned', 
                    'agency_name', 'residence_type', 'isverified']

date_cols = ['reactivatedat']

id_cols = ['id', 'name', 'link', 'building_no', 'postal_code','deed_number', 'additional_no']

In [7]:
#her Numerical sütunun min ve max deyerini, deyerlerin sayını ve missing deyerlerin sayını gösterir.  
num_summary = df[num_cols].describe().loc[['min', 'max', 'count']]

num_summary.loc['missing'] = df[num_cols].isnull().sum()

num_summary


,price,rooms,beds,bath,area_m2,lat,long
min,350.0,1.0,0.0,0.0,24.73,21.411700,38.95625
max,13000000.0,345.0,11.0,11.0,103104.38,25.000033,46.98135
count,18315.0,18315.0,18315.0,18315.0,18315.00,18315.000000,18315.00000
missing,0.0,0.0,0.0,0.0,0.00,0.000000,0.00000


In [8]:
#categorical sütunların missing deyerlerinin sayını, benzersiz deyerlerin sayını ve hemin sütunda ən çox olan deyeri gösterir
cat_summary = pd.DataFrame({
    'missing': df[cat_cols].isnull().sum(),
    'unique_count': df[cat_cols].nunique(),
    'frequency': df[cat_cols].mode().iloc[0]
})

cat_summary

,missing,unique_count,frequency
location,0,157,Al Mahdiyah
amenities,68,3239,"Electricity, Water Supply, Sewerage"
furnished,0,2,unfurnished
city,0,2,Riyadh
region,17,14,North Jeddah
district,273,148,Al Mahdiyah
type,1285,2,agency
listing_age,0,13,New
listing_face,9121,10,Eastern
is_listing_constrained,0,2,No


In [9]:

def get_h3_index(row):
    return h3.latlng_to_cell(row['lat'], row['long'], res=10)

df['h3_index'] = df.apply(get_h3_index, axis=1)

df['h3_11'] = df.apply(
    lambda row: h3.latlng_to_cell(
        row['lat'],
        row['long'],
        11
    ),
    axis=1
)

df

,id,name,price,rooms,bath,area_m2,location,lat,long,building_no,amenities,furnished,reactivatedat,city,region,district,link,type,beds,postal_code,additional_no,isverified,listing_age,listing_face,deed_number,is_listing_constrained,is_listing_pawned,agency_name,residence_type,h3_index,h3_11
0,87788343,Luxury property near King Road in Jeddah - Al ...,1230000.0,5,4,186.06,Al Nahdah,21.608896,39.128730,3105,"Electricity, Sewerage, Air Conditioning, Eleva...",unfurnished,2025-10-16,Jeddah,North Jeddah,Al Nahdah,https://www.bayut.sa/en/property/details-87788...,agency,5,23523,8047,True,New,Eastern,260001436485,No,No,Elite Buildings Real Estate Development Company,NaN,8a53a952c68ffff,8b53a952c68dfff
1,87860369,Apartment in North Riyadh，Al Arid 3 bedrooms 1...,1590000.0,3,4,178.04,Al Arid,24.873473,46.611978,7747,"Electricity, Water Supply, Flood Drainage, Fib...",unfurnished,2026-01-02,Riyadh,North Riyadh,Al Arid,https://www.bayut.sa/en/property/details-87860...,agency,3,13337,4558,True,New,Western,5853649382900005,No,No,Al Orouba Real Estate Company,NaN,8a537379e637fff,8b537379e634fff
2,87839439,Apartment with 3 bedrooms for sale in Al Qudsi...,890000.0,3,3,157.12,Al Qadisiyah,24.825030,46.804521,8066,"Electricity, Water Supply, Air Conditioning, S...",unfurnished,2025-12-16,Riyadh,East Riyadh,Al Qadisiyah,https://www.bayut.sa/en/property/details-87839...,agency,3,13252,3068,True,3 years,NaN,317811004196,Yes,No,Wiyyana,family,8a537361bab7fff,8b537361bab0fff
3,87837487,"Apartment for sale in Al Malqa, North Riyadh",1360000.0,2,3,137.98,Al Malqa,24.813335,46.595210,8967,"Electricity, Water Supply, Sewerage, Balcony o...",unfurnished,2025-12-14,Riyadh,North Riyadh,Al Malqa,https://www.bayut.sa/en/property/details-87837...,agency,3,13525,5078,True,5 years,NaN,4347125507700007,No,No,Wiyyana,family,8a5354b4d807fff,8b5354b4d800fff
4,87818947,"Luxury property near the park, prime location ...",850000.0,6,5,210.53,Al Fayhaa,21.502377,39.229822,3106,"Electricity, Elevators in Building, ATM Facili...",unfurnished,2025-11-17,Jeddah,North Jeddah,Al Fayhaa,https://www.bayut.sa/en/property/details-87818...,agency,6,22251,8078,True,New,NaN,325510001720,No,No,Elite Buildings Real Estate Development Company,NaN,8a53a9551247fff,8b53a955125cfff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18310,87836178,"Studio Apartment For Sale in Al Fayhaa, Jeddah",190000.0,2,0,85.00,Al Fayhaa,21.502470,39.232884,8075,NaN,unfurnished,2025-12-12,Jeddah,North Jeddah,Al Fayhaa,https://www.bayut.sa/en/property/details-87836...,agency,0,22251,3418,False,New,Eastern,860002738673,No,No,Altaj Alraqi Real Estate Services Establishment,family,8a53a955c507fff,8b53a955c506fff
18311,87836139,"Studio Apartment For Sale in Al Fayhaa, Jeddah",180000.0,2,0,85.00,Al Fayhaa,21.502470,39.232884,8075,NaN,unfurnished,2025-12-12,Jeddah,North Jeddah,Al Fayhaa,https://www.bayut.sa/en/property/details-87836...,agency,0,22251,3418,False,New,Eastern,860002738673,No,No,Altaj Alraqi Real Estate Services Establishment,family,8a53a955c507fff,8b53a955c506fff
18312,87835839,"Studio Apartment For Sale in Al Fayhaa, Jeddah",180000.0,2,0,85.00,Al Fayhaa,21.502470,39.232884,8075,NaN,unfurnished,2025-12-12,Jeddah,North Jeddah,Al Fayhaa,https://www.bayut.sa/en/property/details-87835...,agency,0,22251,3418,False,New,Eastern,860002738673,No,No,Altaj Alraqi Real Estate Services Establishment,family,8a53a955c507fff,8b53a955c506fff
18313,87818991,"Studio Apartment For Sale in Al Marwah, Jeddah",630000.0,5,0,131.14,Al Marwah,21.626502,39.185383,8948,NaN,unfurnished,2025-11-18,Jeddah,North Jeddah,Al Marwah,https://www.bayut.sa/en/property/details-87818...,NaN,0,23542,2342,False,1 year,Southeasterly,482011000419,No,No,NaN,family,8a53a9cd840ffff,8b53a9cd840afff


In [10]:
# Bilimsel Global Filtreler (Fiyat, Oda, Alan)
# 1. Oda: 1-15 (Apartment icin normal aralik)
# 2. Fiyat: 2000 - 3M SAR (Cok ucuz veya pahali aykiri degerleri atar)
# 3. Alan: 30 - 2000 m2 (Asiri kucuk veya arsa buyuklugundeki yerleri atar)
df = df[
    (df['rooms'].between(1, 15)) & 
    (df['price'].between(2000, 3000000)) & 
    (df['area_m2'].between(30, 2000))
]

# 1. Her benzersiz tapu numarasına bir Property ID ver
# df['property_id'] = df.groupby('deed_number').ngroup()
# Aynı bina + aynı oda sayısı = aynı daire tipi
df['property_id'] = df.groupby(['deed_number', 'rooms', 'h3_11']).ngroup()


# 2. Her mülk için toplam ilan sayısını hesapla (Kaç tane mükerrer olduğunu görmek için)
df['total_listings_count'] = df.groupby('property_id')['id'].transform('count')

# df

C:\Users\ACER\AppData\Local\Temp\ipykernel_22888\3927267550.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['property_id'] = df.groupby(['deed_number', 'rooms', 'h3_11']).ngroup()
C:\Users\ACER\AppData\Local\Temp\ipykernel_22888\3927267550.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_listings_count'] = df.groupby('property_id')['id'].transform('count')


In [11]:
# Her mülk grubu için 'doğru' kabul edilecek değerleri (medyan) hesapla
# Bu işlem uç değerleri (345 oda vb.) otomatik olarak etkisiz hale getirir
group_medians = df.groupby('property_id')[num_cols].transform('median')



# Mevcut değerler medyan ile uyuşuyor mu? (Tolerans: Alan için %10, diğerleri birebir aynı olmalı)
df['is_faulty'] = (
    # (df['rooms'] != group_medians['rooms']) | 
    (df['area_m2'] > group_medians['area_m2'] * 1.1) | 
    (df['area_m2'] < group_medians['area_m2'] * 0.9)
)



print(f"Hatalı/Yalan bilgi içeren ilan sayısı: {df['is_faulty'].sum()}")
df.describe()
# df


Hatalı/Yalan bilgi içeren ilan sayısı: 23


C:\Users\ACER\AppData\Local\Temp\ipykernel_22888\3278611708.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_faulty'] = (


,price,rooms,bath,area_m2,lat,long,reactivatedat,beds,property_id,total_listings_count
count,1.785500e+04,17855.0,17855.000000,17855.000000,17855.000000,17855.000000,17855,17855.000000,17855.000000,17855.000000
mean,7.851611e+05,3.681994,2.202688,260.917268,23.194899,43.014580,2025-10-13 05:36:23.388406528,2.479474,3763.775077,10.178381
min,1.800000e+04,1.0,0.000000,42.860000,21.411700,38.956250,2025-01-12 00:00:00,0.000000,0.000000,1.000000
25%,5.600000e+05,3.0,0.000000,122.190000,21.588321,39.164411,2025-09-01 00:00:00,0.000000,1857.000000,1.000000
50%,7.500000e+05,4.0,3.000000,145.550000,24.507089,46.499509,2025-10-24 00:00:00,3.000000,3912.000000,4.000000
75%,9.500000e+05,4.0,3.000000,203.160000,24.663642,46.630659,2025-12-12 00:00:00,4.000000,5682.500000,13.000000
max,3.000000e+06,13.0,11.000000,2000.000000,25.000033,46.981350,2026-01-07 00:00:00,11.000000,7421.000000,94.000000
std,3.259178e+05,1.047285,1.561112,281.206814,1.551378,3.735457,NaN,1.821056,2177.019801,15.239988


In [12]:
# Her mülk içindeki ilanları tarihe göre sırala ve en güncelini 'Ana İlan' seç
df['is_main_listing'] = False
main_indices = df.sort_values(['property_id', 'reactivatedat'], ascending=[True, False]).groupby('property_id').head(1).index
df.loc[main_indices, 'is_main_listing'] = True
# df

C:\Users\ACER\AppData\Local\Temp\ipykernel_22888\1744022248.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_main_listing'] = False


In [13]:
# Sadece ana ilanları al ve hatalı işaretlenenleri (outliers) dışarıda bırak
df_clean = df[(df['is_main_listing'] == True) & (df['is_faulty'] == False)].copy()

# Temizlik sonrası veri kontrolü
print(f"Başlangıç ilan sayısı: {len(df)}")
print(f"Temizlik sonrası gerçek mülk sayısı: {len(df_clean)}")
print(f"Yeni Maksimum Oda Sayısı: {df_clean['rooms'].max()}") # Artık 345 görmemelisin
df_clean.describe()
# df_clean

Başlangıç ilan sayısı: 17855
Temizlik sonrası gerçek mülk sayısı: 7417
Yeni Maksimum Oda Sayısı: 13


,price,rooms,bath,area_m2,lat,long,reactivatedat,beds,property_id,total_listings_count
count,7.417000e+03,7417.0,7417.000000,7417.000000,7417.000000,7417.000000,7417,7417.000000,7417.000000,7417.000000
mean,8.244159e+05,3.826075,2.320615,220.356021,23.449259,43.619406,2025-10-13 08:46:31.856545792,2.619523,3708.867736,2.405959
min,2.500000e+04,1.0,0.000000,42.860000,21.411700,38.956250,2025-01-20 00:00:00,0.000000,0.000000,1.000000
25%,5.700000e+05,3.0,0.000000,120.980000,21.606991,39.198249,2025-08-27 00:00:00,0.000000,1854.000000,1.000000
50%,7.400000e+05,4.0,3.000000,146.200000,24.554096,46.554893,2025-10-27 00:00:00,3.000000,3708.000000,1.000000
75%,1.000000e+06,5.0,3.000000,190.100000,24.807567,46.751501,2025-12-12 00:00:00,4.000000,5563.000000,2.000000
max,3.000000e+06,13.0,11.000000,2000.000000,25.000033,46.981350,2026-01-07 00:00:00,11.000000,7421.000000,94.000000
std,3.721518e+05,1.149764,1.595976,230.313300,1.541328,3.702728,NaN,1.907527,2142.469781,4.325931


In [14]:
# # Bilimsel ve piyasa gerçeklerine dayalı nihai filtreleme
# df_clean = df_clean[
#     (df_clean['price'] >= 100000) &      # 100k altı satış verisi için gerçekçi değildir
#     (df_clean['rooms'] <= 15) &         # 15 odadan fazlası 'Apartment' kategorisi için hatadır
#     (df_clean['area_m2'] >= 20) &       # 20 m2 altı ev değildir
#     (df_clean['area_m2'] <= 1500)       # 1500 m2 üstü genelde hatalı arsa verisidir
# ].copy()

# print(f"Global temizlik sonrası kalan mülk sayısı: {len(df_clean)}")
# print(f"Yeni Max Oda: {df_clean['rooms'].max()} | Yeni Min Fiyat: {df_clean['price'].min()}")
# df_clean.describe()


In [15]:
# 1. Önce veriyi tarihe göre sıralayalım ki 'drop_duplicates' en güncel ilanı tutsun
df_final = df_clean.sort_values(by='reactivatedat', ascending=False)

# 2. Hem Hard Match hem Soft Match'i kapsayan nihai tekilleştirme
# Aynı konum, aynı m2 ve aynı oda sayısına sahip her şeyi teke indiriyoruz
df_final = df_final.drop_duplicates(
    subset=['h3_index', 'area_m2', 'rooms', 'bath'], 
    keep='first'
)

print(f"Temizlik Öncesi (Ham): 18.315")
print(f"Hard Match + Outlier Sonrası: {len(df_clean)}") 
print(f"Soft Match (NİHAİ) Sonrası: {len(df_final)}")
# df_final


Temizlik Öncesi (Ham): 18.315
Hard Match + Outlier Sonrası: 7417
Soft Match (NİHAİ) Sonrası: 7023


In [16]:
# --- NİHAİ SÜTUN SIRALAMASI ---
desired_cols = [
    'price', 'rooms', 'beds', 'area_m2', 'deed_number', 'building_no', 
    'additional_no', 'postal_code', 'lat', 'long', 'property_id', 'total_listings_count', 'is_faulty', 'is_main_listing', 'link', 
    'id', 'h3_11', 'name', 'bath', 'location', 'city', 'region', 'district', 
    'reactivatedat', 'listing_age', 'listing_face', 'type', 'isverified', 
    'furnished', 'amenities', 'agency_name', 'residence_type', 'is_listing_constrained', 'is_listing_pawned', 'h3_index'
]

# Mevcut olmayan sütunları (varsa) listeden çıkaralım
existing_cols = [c for c in desired_cols if c in df_final.columns]
# Listede olmayan ama df'de olan sütunları sona ekleyelim
other_cols = [c for c in df_final.columns if c not in existing_cols]

df_final = df_final[existing_cols + other_cols]

print("Sütunlar başarıyla yeniden sıralandı.")
df_final.head()

df_final



Sütunlar başarıyla yeniden sıralandı.


,price,rooms,beds,area_m2,deed_number,building_no,additional_no,postal_code,lat,long,property_id,total_listings_count,is_faulty,is_main_listing,link,id,h3_11,name,bath,location,city,region,district,reactivatedat,listing_age,listing_face,type,isverified,furnished,amenities,agency_name,residence_type,is_listing_constrained,is_listing_pawned,h3_index
1492,880000.0,4,4,127.42,5878890131400010,2322,7422,13754,24.654275,46.515877,4311,2,False,True,https://www.bayut.sa/en/property/details-87862...,87862939,8b5354a5429efff,Apartment for sale in Al-Mahdiyah neighborhood,3,Al Mahdiyah,Riyadh,West Riyadh,Al Mahdiyah,2026-01-07,New,Southern,developer,False,unfurnished,"Electricity, Water Supply, Sewerage, Fiber Opt...",MUAFQA REAL ESTATE COMPANY,NaN,No,No,8a5354a5429ffff
950,1200000.0,6,6,230.56,860002610239,7783,3015,23525,21.609659,39.147102,6106,1,False,True,https://www.bayut.sa/en/property/details-87862...,87862909,8b53a952d4c5fff,Spacious apartments with a terrace for each ap...,4,Al Salamah,Jeddah,North Jeddah,Al Salamah,2026-01-07,New,NaN,agency,False,unfurnished,"Private Entrance, Electricity, Water Supply, S...",Almurad real estate,NaN,No,No,8a53a952d4c7fff
1693,650000.0,4,4,111.49,160002909846,7984,3494,23437,21.592591,39.157788,266,28,False,True,https://www.bayut.sa/en/property/details-87862...,87862817,8b53a9529096fff,"4 Bedroom Apartment For Sale in Al Salam, Jeddah",3,Al Salamah,Jeddah,North Jeddah,Al Salamah,2026-01-07,New,NaN,agency,False,unfurnished,"Electricity, Water Supply, Sewerage",Fikr Al E'mar Company,family,No,No,8a53a9529097fff
1694,780000.0,4,3,132.26,960001396609,7873,3832,23433,21.565379,39.160321,6737,6,False,True,https://www.bayut.sa/en/property/details-87862...,87862815,8b53a950db26fff,"4 Bedroom Apartment For Sale in Al Rawda, Jeddah",2,Al Rawdah,Jeddah,North Jeddah,Al Rawdah,2026-01-07,New,NaN,agency,False,unfurnished,"Electricity, Water Supply, Sewerage",Fikr Al E'mar Company,family,No,No,8a53a950db27fff
1695,670000.0,4,4,200.13,260026300383,2880,6574,14743,24.513676,46.729632,836,9,False,True,https://www.bayut.sa/en/property/details-87862...,87862735,8b5355c302adfff,4-bedroom apartment for sale in Badr. For comm...,5,Badr,Riyadh,South Riyadh,Badr,2026-01-07,New,Eastern,agency,False,unfurnished,"Electricity, Water Supply, Sewerage",Wathaek Real Estate Development Company,family,No,No,8a5355c302affff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9258,1900000.0,1,1,101.96,311315001904,6635,2523,23415,21.577993,39.110287,1251,1,False,True,https://www.bayut.sa/en/property/details-87612...,87612681,8b53a952a5a6fff,"Apartment for sale in Al Shati, North Jeddah",2,Al Shati,Jeddah,North Jeddah,Al Shati,2025-01-29,10+ years,NaN,agency,False,furnished,"Electricity, Water Supply",Rowad Al Maqayis Real Estate Services Company,family,No,No,8a53a950525ffff
17982,395000.0,3,0,625.00,393813010051,2940,7206,23532,21.624754,39.167826,2534,1,False,True,https://www.bayut.sa/en/property/details-87612...,87612325,8b53a9cde6a8fff,"Apartment For Sale in Al Nuzhah, Jeddah",3,Al Nuzhah,Jeddah,North Jeddah,Al Nuzhah,2025-01-28,New,Southern,NaN,False,unfurnished,"Electricity, Water Supply",NaN,family,No,No,8a53a9cde6affff
3976,680000.0,5,5,188.20,325416007818,8148,3514,23333,21.548797,39.188421,1476,1,False,True,https://www.bayut.sa/en/property/details-87611...,87611952,8b53a9572273fff,"Luxury Apartments for Sale in Al Aziziyah, Nor...",4,Al Aziziyah,Jeddah,North Jeddah,Al Aziziyah,2025-01-27,New,South Western,agency,False,unfurnished,"Electricity, Water Supply, Sewerage, Balcony o...",The Golden Real Estate Deal Office,family,No,No,8a53a9572277fff
5844,640000.0,6,6,208.83,725420011337,7651,3041,23741,21.705628,39.210164,5140,1,False,True,https://www.bayut.sa/en/property/details-87610...,87610606,8b53a9cc576efff,"Apartment for sale in Al Rayaan, North Jeddah",4,Al Rayaan,Jeddah,North Jeddah,Al Rayaan,2025-01-20,New,NaN,agency,False,unf